In [7]:
import numpy as np
import skimage
from skimage import io
from skimage import img_as_ubyte
from matplotlib import pyplot as plt
from skimage import data
from skimage.transform import resize
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

In [8]:
#Gaussian Filter

def gaussian_kernel(k_size, sigma):
    size = k_size//2
    y, x = np.ogrid[-size:size+1, -size:size+1]
    filter = 1/(2*np.pi * (sigma**2)) * np.exp(-1 *(x**2 + y**2)/(2*(sigma**2)))
    sum = filter.sum()
    filter /= sum
    return filter

def padding(img, k_size):
    pad_size = k_size//2
    h, w, ch = img.shape
    
    res = np.zeros((h + (2*pad_size), w+(2*pad_size), ch), dtype=np.float)
    
    if pad_size == 0:
        res = img.copy()
    else:
        res[pad_size:-pad_size, pad_size:-pad_size] = img.copy()
    return res

def gaussian_filtering(img, k_size=5,sigma=4):
    h, w, ch = img.shape
    filter = gaussian_kernel(k_size, sigma)
    pad_img = padding(img,k_size)
    filtered_img = np.zeros((h, w, ch), dtype=np.float32)
    
    for ch in range(0, ch):
        for i in range(h):
            for j in range(w):
                filtered_img[i, j, ch] = np.sum(filter * pad_img[i:i+k_size, j:j+k_size, ch])

    return filtered_img

In [9]:
def median_filter(img, filter_size=(3, 3), stride=1):
    
    img_shape = np.shape(img) # 이미지 크기 가져오기

    # 결과 이미지의 형태를 계산합니다.
    # 여기서 filter_size와 stride를 고려하여 결과 이미지의 각 차원 크기를 계산합니다.
    result_shape = tuple(np.int64((np.array(img_shape[:2]) - np.array(filter_size)) / stride) + 1) + (img_shape[2],)

    # 결과 이미지를 저장할 배열을 초기화합니다.
    result = np.zeros(result_shape)

    # 이미지를 순회하면서 각 픽셀에 대해 메디안 필터를 적용합니다.
    for h in range(0, result_shape[0], stride):
        for w in range(0, result_shape[1], stride):
            for c in range(img_shape[2]):
                # 현재 위치에서 필터 크기만큼의 영역을 추출합니다.
                tmp = img[h:h + filter_size[0], w:w + filter_size[1], c].ravel()
                
                # 추출된 영역의 값을 정렬합니다.
                tmp = np.sort(tmp)

                # 정렬된 값 중 중앙값을 결과 이미지의 해당 위치에 할당합니다.
                result[h, w, c] = tmp[int(len(tmp) / 2)]

    return result


In [10]:
def fast_bilateral_filter(noisy_img, k_size=5, sigma_space=4, sigma_intensity=0.2):
    h, w, ch = noisy_img.shape
    bilateral_noisy_img = np.zeros((h, w, ch))

    spatial_filter = gaussian_kernel(k_size, sigma_space)

    for c in range(ch):
        intensity_center = noisy_img[:, :, c]
        weighted_sum = np.zeros_like(intensity_center)
        normalization_factor = np.zeros_like(intensity_center)

        for m in range(-k_size//2, k_size//2 + 1):
            for n in range(-k_size//2, k_size//2 + 1):
                i_neighbors = np.clip(np.arange(h) + m, 0, h - 1)
                j_neighbors = np.clip(np.arange(w) + n, 0, w - 1)
                intensity_neighbors = noisy_img[i_neighbors, :, c][:, j_neighbors]
                weight_intensity = np.exp(-(intensity_center - intensity_neighbors)**2 / (2 * sigma_intensity**2))
                weight_spatial = spatial_filter[m + k_size//2, n + k_size//2]
                weighted_sum += intensity_neighbors * weight_intensity * weight_spatial
                normalization_factor += weight_intensity * weight_spatial

        bilateral_noisy_img[:, :, c] = weighted_sum / normalization_factor
    return bilateral_noisy_img

In [11]:
# Image file names
name = ['baby', 'bagles', 'beach', 'book', 'dog', 'girl_ani', 'lego', 'kitty', 'house', 'street']

for fn in name:
    _noisy = '_noisy'
    png = '.png'

    # Load clean and noisy images
    filename = fn + png
    clean_img = io.imread(filename).astype(float)
    clean_img = clean_img[:, :, 0:3] / 255.0

    filename = fn + _noisy + png
    noisy_img = io.imread(filename).astype(float)
    noisy_img = noisy_img[:, :, 0:3] / 255.0

    max_psnr_img = noisy_img
    max_psnr_img_title = filename
    max_psnr = peak_signal_noise_ratio(clean_img, noisy_img)

    max_ssim_img = noisy_img
    max_ssim_img_title = filename
    max_ssim = structural_similarity(clean_img, noisy_img, channel_axis=2, full=False)

    for k in range(0, 2):
        mfs = 3 + (k * 2)
        filter_size = (mfs, mfs)
        median_img = median_filter(noisy_img, filter_size, 1)
        median_img_resized = resize(median_img, clean_img.shape, mode='constant', anti_aliasing=True)
        median_img_resized = np.clip(median_img_resized, 0., 1.0)
        m_psnr = peak_signal_noise_ratio(clean_img, median_img_resized)
        m_ssim = structural_similarity(clean_img, median_img_resized, channel_axis=2, full=False)
        title = "\nmedian filter - stride 1, filter size : " + str(mfs)

        if max_psnr < m_psnr:
            max_psnr_img = median_img_resized
            max_psnr_img_title = filename + title
            max_psnr = m_psnr

        if max_ssim < m_ssim:
            max_ssim_img = median_img_resized
            max_ssim_img_title = filename + title
            max_ssim = m_ssim

        for i in range(0, 3):
            for j in range(0, 3):
                k_size = 3 + 2 * i
                sigma = 3 * j + 1
                res = gaussian_filtering(median_img_resized, k_size=k_size, sigma=sigma)
                g_psnr = peak_signal_noise_ratio(clean_img, res)
                g_ssim = structural_similarity(clean_img, res, channel_axis=2, full=False)

                if max_psnr < g_psnr:
                    max_psnr_img = res
                    max_psnr_img_title = filename + title + "\ngaussian filter - kernel size : " + str(k_size) + ",  sigma = " + str(sigma)
                    max_psnr = g_psnr
                    
                if max_ssim < g_ssim:
                    max_ssim_img = res
                    max_ssim_img_title = filename + title + "\ngaussian filter - kernel size : " + str(k_size) + ",  sigma = " + str(sigma)
                    max_ssim = g_ssim
        
        sigma_space = [3, 5, 7]
        sigma_intensity = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        for s_s in sigma_space:
            for s_i in sigma_intensity:
                bf = fast_bilateral_filter(noisy_img, k_size, s_s, s_i)
                bf_psnr = peak_signal_noise_ratio(clean_img, bf)
                bf_ssim = structural_similarity(clean_img, bf, channel_axis=2, full=False)
                tmptitle = "\nbilateral_filter - kernel size : " + str(k_size) + ", sigma_space : " + str(s_s) + ", sigma_intensity : " + str(s_s)
                
                if max_psnr < bf_psnr:
                    max_psnr_img = bf
                    max_psnr_img_title = filename + tmptitle
                    max_psnr = bf_psnr
                    
                if max_ssim < bf_ssim:
                    max_ssim_img = bf
                    max_ssim_img_title = filename + tmptitle
                    max_ssim = bf_ssim
                    
                mbf = fast_bilateral_filter(median_img_resized, k_size, s_s, s_i)
                mbf_psnr = peak_signal_noise_ratio(clean_img, mbf)
                mbf_ssim = structural_similarity(clean_img, mbf, channel_axis=2, full=False)
                
                if max_psnr < mbf_psnr:
                    max_psnr_img = mbf
                    max_psnr_img_title = filename + title + tmptitle
                    max_psnr = mbf_psnr
                    
                if max_ssim < mbf_ssim:
                    max_ssim_img = mbf
                    max_ssim_img_title = filename + title + tmptitle
                    max_ssim = mbf_ssim
                
    print(filename)
    print(max_psnr_img_title)
    print(str(max_psnr))
    print(max_ssim_img_title)
    print(str(max_ssim))
    plt.imsave('max_psnr' + fn + png, max_psnr_img)
    plt.imsave('max_ssim' + fn + png, max_ssim_img)


/var/folders/n1/t3hm6b612gg69mlfjrdv4ht00000gn/T/ipykernel_3536/3250497239.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res = np.zeros((h + (2*pad_size), w+(2*pad_size), ch), dtype=np.float)
/var/folders/n1/t3hm6b612gg69mlfjrdv4ht00000gn/T/ipykernel_3536/2766140423.py:50: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  g_psnr = peak_signal_noise_ratio(clean_img, res)
